In [132]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import pyspark.sql.types as st
from datetime import date

spark = SparkSession.builder.getOrCreate()

daily_country_confirmed_cases = "./data/Daily_Country_Wise_Confirmed_Cases.csv"

daily_df = spark.read.csv(daily_country_confirmed_cases,inferSchema=True,header=True)


In [133]:
# Change to importing this function
def date_check():
    today = date.today()
    return str(today)

from datetime import datetime, timedelta

def date_minus_weeks(x:int) -> str:
    """
    Returns a timedelta STRING
    """
    return str(date.today() - timedelta(weeks = x))

In [134]:
# Showing the change of cases over a month
# daily_df.select('Country',daily_df[-1]).where(daily_df[-1] >= 1).show()

# Convert to transpose

daily_df.select('*').where(daily_df[-1] >= 1).show()

+-------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|      Country|2022-05-06|2022-05-12|2022-05-13|2022-05-15|2022-05-17|2022-05-18|2022-05-19|2022-05-20|2022-05-21|2022-05-23|2022-05-24|2022-05-25|2022-05-26|2022-05-27|2022-05-28|2022-05-29|2022-05-30|2022-05-31|2022-06-01|2022-06-02|2022-06-03|2022-06-04|2022-06-06|2022-06-07|2022-06-08|2022-06-09|2022-06-10|2022-06-11|2022-06-12|2022-06-13|2022-06-14|2022-06-15|2022-06-16|2022-06-17|2022-06-18|2022-06-19|2022-06-20|2022-06-21|2022-06-22|2022-06-23|2

In [135]:
# Shows the change over 1 week
daily_df.select('Country', daily_df[date_minus_weeks(2)],daily_df[date_minus_weeks(1)]).where(daily_df[date_minus_weeks(1)] > 10).show()

+-------------+----------+----------+
|      Country|2022-06-22|2022-06-29|
+-------------+----------+----------+
|     Portugal|        13|        18|
|United States|        14|        45|
|      Germany|        54|        35|
|       Canada|         0|        12|
|       Brazil|         2|        16|
+-------------+----------+----------+



22/07/07 08:15:59 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 55163239 ms exceeds timeout 120000 ms
22/07/07 08:15:59 WARN SparkContext: Killing executors is not supported by current scheduler.


In [43]:
daily_df.select(
    'Country',
    daily_df[date_minus_weeks(7)],
    daily_df[date_minus_weeks(6)],
    daily_df[date_minus_weeks(5)],
    daily_df[date_minus_weeks(4)],
    daily_df[date_minus_weeks(3)],
    daily_df[date_minus_weeks(2)],
    daily_df[date_minus_weeks(1)])\
        .show()

+--------------------+----------+----------+----------+----------+----------+----------+----------+
|             Country|2022-05-18|2022-05-25|2022-06-01|2022-06-08|2022-06-15|2022-06-22|2022-06-29|
+--------------------+----------+----------+----------+----------+----------+----------+----------+
|             England|         2|         7|         5|         0|         0|         0|         0|
|            Portugal|        11|        10|        19|        25|        10|        13|        18|
|               Spain|         7|         8|         0|        34|         0|         0|         0|
|       United States|         1|         2|         0|         5|        12|        14|        45|
|             Germany|         0|         1|         0|        13|        45|        54|        35|
|             Belgium|         0|         2|         5|         8|        27|         0|         0|
|              Sweden|         0|         1|         0|         0|         4|         0|         0|


In [77]:
# Worldwide total, sum of all.
from functools import reduce

def column_add(a,b):
     return  a.__add__(b)
agg1_df = daily_df.drop('Country')

# def column_add(a:int,b:int):
#      if type(a) == int and type(b) == int:
#           return  a+b
#      else:
#           pass

agg1_df = agg1_df.withColumn('total', reduce(column_add, (agg1_df[col] for col in agg1_df.columns)))
agg1_df.groupBy().sum().show()

# daily_df.withColumn('totals', reduce(column_add, (daily_df[col] for col in daily_df.columns))).show()

# daily_df.drop(daily_df['Country']).withColumn('total_col', reduce(column_add, (daily_df[col] for col in daily_df.columns))).show()



+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+
|sum(2022-05-06)|sum(2022-05-12)|sum(2022-05-13)|sum(2022-05-15)|sum(2022-05-17)|sum(2022-05-18)|sum(2022-05-19)|sum(2022-05-20)|sum(2022-05-21)|sum(2022-05-23)|sum(2022-05-24)|sum(2022-05-25)|sum(2022-05-26)|sum(2022-0

In [222]:
# Create dataframe for region totals daily totals and sums

africa = ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde", "Central African Republic",
    "Chad", "Comoros", "Cote d'Ivoire", "Democratic Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea",
    "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi",
    "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo", "Reunion",
    "Rwanda", "Saint Helena", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "South Sudan", 
    "Sudan", "Swaziland", "Tanzania", "Togo", "Tunisia", "Uganda", "Western Sahara", "Zambia", "Zimbabwe"]

asia = ["Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Burma", "Cambodia", "China", "Cyprus",
    "East Timor", "Georgia", "Hong Kong", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", "Kazakhstan", "Kuwait",
    "Kyrgyzstan", "Laos", "Lebanon", "Macau", "Malaysia", "Maldives", "Mongolia", "Nepal", "North Korea", "Oman", "Pakistan", "Philippines",
    "Qatar", "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Taiwan", "Tajikistan", "Thailand", "Turkey", "Turkmenistan",
    "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen"]

caribbean = ["Anguilla", "Antigua and Barbuda", "Aruba", "The Bahamas", "Barbados", "Bermuda", "British Virgin Islands", "Cayman Islands",
    "Cuba", "Dominica", "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica", "Martinique", "Montserrat", "Netherlands Antilles",
    "Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Trinidad and Tobago", "Turks and Caicos Islands", "U.S. Virgin Islands"]

central_america = ["Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Nicaragua", "Panama"]

europe = ["Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", 
    "Czech Republic", "Denmark", "England", "Estonia", "Finland", "France", "Germany", "Gibraltar", "Greece", "Holy See", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia",
    "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Russia", 
    "San Marino", "Slovak Republic", "Slovenia", "Spain", "Serbia", "Serbia and Montenegro", "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Northern Ireland", "Wales"]

north_america = ["Canada", "Greenland", "Mexico", "Saint Pierre and Miquelon", "United States"]

oceania = ["American Samoa", "Australia", "Christmas Island", "Cocos (Keeling) Islands", "Cook Islands", "Federated States of Micronesia", "Fiji", "French Polynesia", 
    "Guam", "Kiribati", "Marshall Islands", "Nauru", "New Caledonia", "New Zealand", "Niue", "Northern Mariana Islands", "Palau", "Papua New Guinea", "Pitcairn Islands",
    "Samoa", "Solomon Islands", "Tokelau", "Tonga", "Tuvalu", "Vanuatu", "Wallis and Futuna Islands"]

south_america = ["Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Falkland Islands", "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay",
    "Venezuela"]

In [213]:
# ASIA TOTALS

from pyspark.sql.functions import col
from pyspark.sql.types import StringType

# Total daily average. EACH DATE HAS A SUM FUNCTION RAN OVER IT

asia_df = daily_df.filter(daily_df['Country'].isin(asia))

asia_df = asia_df.agg(*[sf.sum(asia_df[c_name]) for c_name in asia_df.columns])

asia_df = asia_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

asia_df = asia_df.na.fill(value='asia_totals')


+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|sum(Country)|sum(2022-05-06)|sum(2022-05-12)|sum(2022-05-13)|sum(2022-05-15)|sum(2022-05-17)|sum(2022-05-18)|sum(2022-05-19)|sum(2022-05-20)|sum(2022-05-21)|sum(2022-05-23)|sum(2022-05-24)|sum(2022-05-25)|sum(2022-05

In [171]:
# Africa
africa_df = daily_df.where(daily_df['Country'].isin(africa))

africa_df = africa_df.agg(*[sf.sum(africa_df[c_name])for c_name in africa_df.columns])

africa_df = africa_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

africa_df = africa_df.na.fill(value='africa_totals')

In [172]:
# Caribbean
caribbean_df = daily_df.where(daily_df['Country'].isin(caribbean))

caribbean_df = caribbean_df.agg(*[sf.sum(caribbean_df[c_name])for c_name in caribbean_df.columns])

caribbean_df = caribbean_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

caribbean_df = caribbean_df.na.fill(value='caribbean_totals')

In [173]:
# Central America

central_america_df = daily_df.where(daily_df['Country'].isin(central_america))

central_america_df = central_america_df.agg(*[sf.sum(central_america_df[c_name])for c_name in central_america_df.columns])

central_america_df = central_america_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

central_america_df = central_america_df.na.fill(value='central_america_totals')

In [224]:
# Europe

europe_df = daily_df.where(daily_df['Country'].isin(europe))

europe_df = europe_df.agg(*[sf.sum(europe_df[c_name])for c_name in europe_df.columns])

europe_df = europe_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

europe_df = europe_df.na.fill(value='europe_totals')

In [176]:
# North America

north_america_df = daily_df.where(daily_df['Country'].isin(north_america))

north_america_df = north_america_df.agg(*[sf.sum(north_america_df[c_name])for c_name in north_america_df.columns])

north_america_df = north_america_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

north_america_df = north_america_df.na.fill(value='north_america_totals')

In [177]:
# Oceania

oceania_df = daily_df.where(daily_df['Country'].isin(oceania))

oceania_df = oceania_df.agg(*[sf.sum(oceania_df[c_name])for c_name in oceania_df.columns])

oceania_df = oceania_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

oceania_df = oceania_df.na.fill(value='oceania_totals')

In [178]:
# South America

south_america_df = daily_df.where(daily_df['Country'].isin(south_america))

south_america_df = south_america_df.agg(*[sf.sum(south_america_df[c_name])for c_name in south_america_df.columns])

south_america_df = south_america_df.withColumn('sum(Country)', col('sum(Country)').cast(StringType()))

south_america_df = south_america_df.na.fill(value='south_america_totals')

In [226]:
# All region dataframe

region_df_list = ['south_america_df', 'oceania_df', 'north_america_df', 'asia_df', 'africa_df', 'caribbean_df', 'europe_df']

region_df = south_america_df.union(oceania_df)

region_df = region_df.union(north_america_df)

region_df = region_df.union(asia_df)

region_df = region_df.union(africa_df)

region_df = region_df.union(caribbean_df)

region_df = region_df.union(europe_df)

region_df = region_df.withColumnRenamed('sum(Country)', 'region')

region_df.toPandas()

,region,sum(2022-05-06),sum(2022-05-12),sum(2022-05-13),sum(2022-05-15),sum(2022-05-17),sum(2022-05-18),sum(2022-05-19),sum(2022-05-20),sum(2022-05-21),...,sum(2022-06-22),sum(2022-06-23),sum(2022-06-24),sum(2022-06-25),sum(2022-06-26),sum(2022-06-27),sum(2022-06-28),sum(2022-06-29),sum(2022-06-30),sum(2022-07-01)
0,south_america_totals,0,0,0,0,0,0,0,0,0,...,2,11,1,2,2,0,4,16,1,0
1,oceania_totals,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,0
2,north_america_totals,0,0,0,0,0,1,2,4,0,...,14,42,28,0,0,65,82,57,44,64
3,asia_totals,0,0,0,0,0,0,0,0,1,...,3,0,4,0,0,13,4,5,5,0
4,africa_totals,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
5,africa_totals,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,europe_totals,1,1,1,4,3,20,14,40,15,...,147,383,288,1,165,289,275,68,758,143


In [228]:
dropped_df = region_df.drop('region')
agg2_df = region_df.withColumn('total', reduce(column_add, (dropped_df[col] for col in dropped_df.columns)))

In [208]:
# agg2_df = region_df.drop('region')
# agg2_df = agg2_df.withColumn('total', reduce(column_add, (region_df[col] for col in agg2_df.columns)))


In [229]:
# daily stats followed by the total of each recorded date
# agg2_df.show()
agg2_df.toPandas()

,region,sum(2022-05-06),sum(2022-05-12),sum(2022-05-13),sum(2022-05-15),sum(2022-05-17),sum(2022-05-18),sum(2022-05-19),sum(2022-05-20),sum(2022-05-21),...,sum(2022-06-23),sum(2022-06-24),sum(2022-06-25),sum(2022-06-26),sum(2022-06-27),sum(2022-06-28),sum(2022-06-29),sum(2022-06-30),sum(2022-07-01),total
0,south_america_totals,0,0,0,0,0,0,0,0,0,...,11,1,2,2,0,4,16,1,0,56
1,oceania_totals,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,1,0,12
2,north_america_totals,0,0,0,0,0,1,2,4,0,...,42,28,0,0,65,82,57,44,64,757
3,asia_totals,0,0,0,0,0,0,0,0,1,...,0,4,0,0,13,4,5,5,0,61
4,africa_totals,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,24
5,africa_totals,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
6,europe_totals,1,1,1,4,3,20,14,40,15,...,383,288,1,165,289,275,68,758,143,5233


In [230]:
#Total for each recorded date and total of totals for the world
world_total_df = agg2_df.groupBy().sum()
world_total_df.toPandas()

,sum(sum(2022-05-06)),sum(sum(2022-05-12)),sum(sum(2022-05-13)),sum(sum(2022-05-15)),sum(sum(2022-05-17)),sum(sum(2022-05-18)),sum(sum(2022-05-19)),sum(sum(2022-05-20)),sum(sum(2022-05-21)),sum(sum(2022-05-23)),...,sum(sum(2022-06-23)),sum(sum(2022-06-24)),sum(sum(2022-06-25)),sum(sum(2022-06-26)),sum(sum(2022-06-27)),sum(sum(2022-06-28)),sum(sum(2022-06-29)),sum(sum(2022-06-30)),sum(sum(2022-07-01)),sum(total)
0,1,1,1,4,3,21,16,46,16,72,...,436,321,3,167,367,366,147,809,207,6144


In [246]:
from pyspark.sql.functions import mean

#Averages of worldwide cases

agg3_df = agg2_df.groupBy().mean()
agg3_pd_df = agg3_df.toPandas()
agg3_pd_df.round()

,avg(sum(2022-05-06)),avg(sum(2022-05-12)),avg(sum(2022-05-13)),avg(sum(2022-05-15)),avg(sum(2022-05-17)),avg(sum(2022-05-18)),avg(sum(2022-05-19)),avg(sum(2022-05-20)),avg(sum(2022-05-21)),avg(sum(2022-05-23)),...,avg(sum(2022-06-23)),avg(sum(2022-06-24)),avg(sum(2022-06-25)),avg(sum(2022-06-26)),avg(sum(2022-06-27)),avg(sum(2022-06-28)),avg(sum(2022-06-29)),avg(sum(2022-06-30)),avg(sum(2022-07-01)),avg(total)
0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,7.0,2.0,10.0,...,62.0,46.0,0.0,24.0,52.0,52.0,21.0,116.0,30.0,878.0
